In [ ]:
import pandas as pd
import datetime

# Bibliotecas Google Golab
from google.colab import auth
auth.authenticate_user()
# Importar biblioteca biqquery
from google.cloud import bigquery
# Bibliotecas Google Sheet
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)
from gspread_dataframe import set_with_dataframe
# Montar Drive
from google.colab import drive
drive.mount('/content/drive')
# Classe dados 
!cp /content/drive/MyDrive/Colab_Notebooks/pacotes_modulos/modulos_colab.py /content
from modulos_colab import dados
!cp /content/drive/MyDrive/Colab_Notebooks/Autenticacoes/chaves_tokens.py /content
%run chaves_tokens.py
!cp /content/drive/MyDrive/Colab_Notebooks/pacotes_modulos/CommunicationCreator.py /content
from CommunicationCreator import CommunicationCreator



# GFL Magalu

In [ ]:

def base_gfl(arquivo_gfl:str)->pd.DataFrame:
    """
    Parâmetro:
    arquivo_gfl: Arquivo original GFL
    _______________________________________
    Função para tratar os dados da base GFL, criar colunas: diferença frete e delta.
    """
    df_gfl_original = pd.read_excel(arquivo_gfl, usecols=["Nro. Pedido",
                                                            "Cod. Remetente",
                                                            "Valor Frete Peso Atual",
                                                            "Valor Frete Peso Recalculado",
                                                            "Peso taxado Atual",
                                                            "Peso taxado Recalculado",
                                                            "Peso Informado",
                                                            "Peso Aferido",
                                                            "Cubagem Informada",
                                                            "Cubagem Aferida"])

    df_gfl_original.rename(columns={"Nro. Pedido":"pedido",
                                    "Cod. Remetente":"seller"}, inplace=True)
    df_gfl_original.columns = df_gfl_original.columns.str.lower().str.replace(" ","_")
    df_gfl_original["pedido"] = df_gfl_original["pedido"].astype(str).str.strip()
    df_gfl_original["diferenca_frete"] = round(df_gfl_original["valor_frete_peso_recalculado"]-df_gfl_original["valor_frete_peso_atual"],2)
    df_gfl_original["delta"] = round(df_gfl_original["peso_taxado_recalculado"]-df_gfl_original["peso_taxado_atual"],2)

    return df_gfl_original



def gfl_magalu(df) -> pd.DataFrame:
    """
    Parâmetro:
    DF: Dataframe GFL tratado
    __________________________________________
    Função para filtrar somente pedidos magalu 3P e excluir frete 0.
    """
    df_gfl_original = df
    df_gfl_original['val_pedido'] = df_gfl_original["pedido"].str.findall(r'\d{16}')
    df_gfl_original = df_gfl_original.explode('val_pedido').fillna("-").drop_duplicates()
    df_gfl_magalu = df_gfl_original.query('val_pedido!="-"')
    df_gfl_magalu = df_gfl_magalu.query('diferenca_frete>0.0 & seller!="magazineluiza"')
    df_gfl_magalu.drop(columns=['val_pedido'], inplace=True)
    df_gfl_magalu["pedido"] = df_gfl_magalu["pedido"].str.strip()

    return df_gfl_magalu




def cluster(df) -> pd.DataFrame:
    """
    Parâmetro:
    DF: DataFrame
    _______________________________________________________________________
    Função com o objetivo:
        - Clusterizar os pedidos
        - Criar coluna cluster_de_delta.
    """
    def f_cluster(df):
        if df['delta'] <= 1.00:
            return '0 a 1' 
        elif df['delta'] <= 2.00:
            return '1 a 2'
        elif df['delta'] <= 3.00:
            return '2 a 3'
        elif df['delta'] <= 4.00:
            return '3 a 4'
        elif df['delta'] <= 5.00:
            return '4 a 5'
        elif df['delta'] <= 6.00:
            return '5 a 6'
        elif df['delta'] <= 7.00:
            return '6 a 7'
        elif df['delta'] <= 8.00:
            return '7 a 8'
        elif df['delta'] <= 9.99:
            return '8 a 9'
        elif df['delta'] >= 10.00:
            return '10 a >'
        else:
            return 'verificar'
    
    df['cluster_de_delta'] = df.apply(f_cluster, axis=1)
    return df




def clear_one_gfl(df) -> pd.DataFrame:
    """
    Parâmetro:
        df: DataFrame
    _________________________
    Função com o objetivo:
        - Criar coluna de origem
        - Organizar a ordem das colunas
    """
    df['origem'] = 'gfl'
    df = df[["pedido",
             "seller",
             "peso_informado",
             "peso_aferido",
             "cubagem_informada",
             "cubagem_aferida",
             "peso_taxado_atual",
             "peso_taxado_recalculado",
             "diferenca_frete",
             "delta",
             "cluster_de_delta",
             "origem"]]
    return df





# function execution
df_gfl_original = base_gfl("/content/drive/MyDrive/dados_mle/arquivos_gfl/Base 3P 02.06 a 04.06.xlsx")

df_gfl_magalu = gfl_magalu(df_gfl_original)

df_gfl_magalu = cluster(df_gfl_magalu)

df_gfl_magalu_final = clear_one_gfl(df_gfl_magalu)


# Correios

In [ ]:
def sellers_comunicados(arquivo_d_1:str) -> pd.DataFrame:
    """
    Parâmetro: 
        project: str
        query: str
    ______________________________________________________________________
    Função com o objetivo:
        - Consultar BigQuery os sellers já notificados
    """
    df_dados_ontem = pd.read_excel(arquivo_d_1, sheet_name='Base', usecols=["pedido_seller"])

    df_dados_ontem['pedido_seller_check'] = df_dados_ontem['pedido_seller'].str.strip()
    
    return df_dados_ontem



def base_mail(arquivo_correios:str, pedido_seller_comunicados:pd.DataFrame) -> pd.DataFrame:
    """
    Parâmetro:
    arquivo_correios:str
    _________________________
    Função com o objetivo: de 
        - Tratar os dados da base dos correios
        - Excluir sellers já comunicados, dia anterior   
    """
    df_correios_original = pd.read_excel(arquivo_correios, sheet_name='Base', usecols=["pedido_seller",
                                                                                        "Cluster de Delta",
                                                                                        "diferença_de_frete",
                                                                                        "M_peso",
                                                                                        "C_peso",
                                                                                        "M_peso_cubado",
                                                                                        "C_peso_cubado",
                                                                                        "M_maior",
                                                                                        "C_maior",
                                                                                        "Delta"])
    
    df_correios_original = df_correios_original.merge(pedido_seller_comunicados, how="left", on=["pedido_seller"]).fillna("-")    
    df_correios_original = df_correios_original.query("pedido_seller_check=='-'")
    df_correios_original.drop(columns=['pedido_seller_check'], inplace=True)
    df_correios_original.columns = df_correios_original.columns.str.replace(' ','_').str.lower()
    df_correios_original.rename(columns={"diferença_de_frete":"diferenca_frete",
                                         "m_peso":"peso_informado",
                                         "c_peso":"peso_aferido",
                                         "m_peso_cubado":"cubagem_informada",
                                         "c_peso_cubado":"cubagem_aferida",
                                         "m_maior":"peso_taxado_atual",
                                         "c_maior":"peso_taxado_recalculado",
                                         "Delta":"delta"}, inplace=True)
    
    sep = df_correios_original["pedido_seller"].str.split("_", n = 1, expand = True)
    df_correios_original["pedido"]= sep[0].astype(str)
    df_correios_original["seller"]= sep[1]

    df_correios_original.drop(columns=['pedido_seller'],inplace=True)

    return df_correios_original





def clear_one_mail(df) -> pd.DataFrame:
    """
    Parâmetro:
     DF: Dataframe
    _________________________
    Função com objetivo:
     - Excluir pedidos com status desconsiderar 
     - Excluir diferença maior 0.00
     - Criar coluna origem
    """
    df = df.query("cluster_de_delta!='desconsiderar' and diferenca_frete>0.0")
    df['origem'] = 'correios'
    return df



def clear_two_mail(df) -> pd.DataFrame:
    """
    Parâmetro:
     DF: DataFrame
    _________________________
    Função com objetivo:
     - Reorganizar colunas do DataFrame
    """
    df = df[["pedido",
             "seller",
             "peso_informado",
             "peso_aferido",
             "cubagem_informada",
             "cubagem_aferida",
             "peso_taxado_atual",
             "peso_taxado_recalculado",
             "diferenca_frete",
             "delta",
             "cluster_de_delta",
             "origem"]]
    return df






# function execution
pedido_seller_comunicados = sellers_comunicados("/content/drive/MyDrive/dados_mle/arquivos_correios/Correios | Dispersão de frete/Dispersão Correios_ 01 de Junho.xlsx")


df_correios_original = base_mail("/content/drive/MyDrive/dados_mle/arquivos_correios/Correios | Dispersão de frete/Dispersão Correios_ 01 a 04 Junho.xlsx", pedido_seller_comunicados)


df_correios = clear_one_mail(df_correios_original)


df_correios = clear_two_mail(df_correios)


# Integração Bases - GFL Magalu + Correios

In [ ]:

def integracao_bases(df1,df2):
    """
    Parâmetro:
     DF1: DataFrame GFL Magalu
     DF2: DataFrame Correios Magalu
    _________________________
    Função com objetivo:
     - Unir as bases GFL + Correios
     - Round nas colunas Float
     - Ingestão Tabela BQ
    """
    df_total = pd.concat([df1,df2]).drop_duplicates()
    df_total['pedido'] = df_total['pedido'].astype(str)
    df_total[['peso_informado',
            "peso_aferido",
            "cubagem_informada",
            "cubagem_aferida",
            "peso_taxado_atual",
            "peso_taxado_recalculado",
            "diferenca_frete",
            "delta"]] = round(df_total[['peso_informado',
                                        "peso_aferido",
                                        "cubagem_informada",
                                        "cubagem_aferida",
                                        "peso_taxado_atual",
                                        "peso_taxado_recalculado",
                                        "diferenca_frete",
                                        "delta"]],2)
            
      
    df_total = df_total.astype(str)
    df_total.to_gbq(destination_table='marketplace_analytics.sku_cubagem_comunicado_correios', project_id='maga-bigdata', if_exists='replace')
    
    return print("Tabela Atualizada!")


# function execution
integracao_bases(df_gfl_magalu_final,df_correios)


# Consulta BigQuery

In [ ]:
def query_bq(project:str,query:str) -> pd.DataFrame:
    """
    Parâmetro:
     project:str
     query:str
    __________________________________________________
    Função com objetivo:
     - Query no BQ para pegar os dados do pedido e sku 
    """
    df_bq = dados.consulta_bigquery(project,query)

    df_bq[['PRECO_POR',
        'media_preco',
        'peso_informado',
            'peso_aferido',
            'cubagem_informada',
            'cubagem_aferida',
            'peso_taxado_atual',
            'peso_taxado_recalculado']] = df_bq[['PRECO_POR',
                                                'media_preco',
                                                'peso_informado',
                                                'peso_aferido',
                                                'cubagem_informada',
                                                'cubagem_aferida',
                                                'peso_taxado_atual',
                                                'peso_taxado_recalculado']].astype(str)

    dados.grava_tabela_drive("Base_gfl_correios", 
                            "base_analitica",
                            df_bq)
    return df_bq



# function execution
df_bq = query_bq('maga-bigdata',
                                """
                                    SELECT
                                        DISTINCT 
                                        c.seller,
                                        c.pedido,
                                        c.peso_informado,
                                        c.peso_aferido,
                                        c.cubagem_informada,
                                        c.cubagem_aferida,
                                        c.peso_taxado_atual,
                                        c.peso_taxado_recalculado,
                                        c.cluster_de_delta,
                                        (c.diferenca_frete) AS diferenca_frete,
                                        v.produtomarketplacekey,
                                        s.Carteira,
                                        s.Data_Publicado,
                                        s.Plataforma,
                                        ca.title,
                                        ca.sku,
                                        (ca.qtd_tt_sku) AS qtd_tt_sku,
                                        (ca.qtd_sku_ativo) AS qtd_sku_ativo,
                                        IF
                                        (status_sku=TRUE,'Ativo','Inativo') AS status_sku,
                                        q.maas_publicado AS status_qualidade,
                                        h.status_loja,
                                        e.PRECO_POR,
                                        p.media_preco,
                                        c.origem
                                        FROM
                                        `marketplace_analytics.sku_cubagem_comunicado_correios` AS c
                                        LEFT JOIN (
                                        SELECT
                                            DISTINCT VCAP.NUMEROPEDIDOCLIENTE,
                                            VCAP.IDSELLERMARKETPLACE AS IDSELLERMARKETPLACE,
                                            VCAP.produtomarketplacekey AS produtomarketplacekey,
                                        FROM
                                            maga-bigdata.apolo.vw_fact_vendas_captadas VCAP
                                        WHERE
                                            DATE(VCAP.datapagamentomktp) >= CURRENT_DATE()-45
                                            AND UPPER(VCAP.STATUS) <> 'TESTE'
                                            AND UPPER (IFNULL(VCAP.IDSELLERMARKETPLACE,'-1')) <> 'MAGAZINELUIZA'
                                        GROUP BY
                                            1,
                                            2,
                                            3) AS v
                                        ON
                                        LOWER(c.seller)=LOWER(v.IDSELLERMARKETPLACE)
                                        AND c.pedido=v.NUMEROPEDIDOCLIENTE
                                        LEFT JOIN (
                                        SELECT
                                            DISTINCT SellerID,
                                            Data_Publicado,
                                            Plataforma,
                                            Carteira
                                        FROM
                                            `maga-bigdata.bi_ecomm.TB_SafraSellers` ) AS s
                                        ON
                                        LOWER(s.SellerID)=LOWER(c.seller)
                                        LEFT JOIN (
                                        SELECT
                                            DISTINCT c.seller_id,
                                            c.navigation_id,
                                            c.sku,
                                            c.title,
                                            c.active AS status_sku,
                                            ca.qtd_tt_sku,
                                            cat.qtd_sku_ativo
                                        FROM
                                            `maga-bigdata.catalogo.visao_catalogo_produto_atual` AS c
                                        LEFT JOIN (
                                            SELECT
                                            seller_id,
                                            COUNT( DISTINCT sku) AS qtd_tt_sku
                                            FROM
                                            `maga-bigdata.catalogo.visao_catalogo_produto_atual`
                                            GROUP BY
                                            1) AS ca
                                        ON
                                            c.seller_id=ca.seller_id
                                        LEFT JOIN (
                                            SELECT
                                            seller_id,
                                            COUNT( DISTINCT sku) AS qtd_sku_ativo
                                            FROM
                                            `maga-bigdata.catalogo.visao_catalogo_produto_atual`
                                            WHERE
                                            active=TRUE
                                            GROUP BY
                                            1) AS cat
                                        ON
                                            cat.seller_id=c.seller_id) AS ca
                                        ON
                                        ca.seller_id=v.IDSELLERMARKETPLACE
                                        AND ca.navigation_id=v.produtomarketplacekey
                                        LEFT JOIN (
                                        SELECT
                                            DISTINCT SellerID,
                                            Status AS status_loja
                                        FROM
                                            `maga-bigdata.marketplace_analytics.view_dados_sellers_helena`) AS h
                                        ON
                                        LOWER(h.SellerID)=LOWER(c.seller)
                                        LEFT JOIN (
                                        SELECT
                                            DISTINCT SELLERID,
                                            SKUMAGAZINELUIZA,
                                            ROUND(PRECO_POR,2) AS PRECO_POR
                                        FROM
                                            `maga-bigdata.bi_ecomm.TB_ESTOQUE_3P`) AS e
                                        ON
                                        LOWER(e.SELLERID)=LOWER(v.IDSELLERMARKETPLACE)
                                        AND e.SKUMAGAZINELUIZA=v.produtomarketplacekey
                                        LEFT JOIN (
                                        SELECT
                                            DISTINCT ct.seller_id,
                                            ROUND(AVG(PRECO_POR),2) AS media_preco
                                        FROM
                                            `maga-bigdata.catalogo.visao_catalogo_produto_atual` AS ct
                                        LEFT JOIN (
                                            SELECT
                                            DISTINCT SELLERID,
                                            SKUMAGAZINELUIZA,
                                            PRECO_POR
                                            FROM
                                            `maga-bigdata.bi_ecomm.TB_ESTOQUE_3P` ) AS es
                                        ON
                                            ct.seller_id=es.SELLERID
                                            AND ct.navigation_id=es.SKUMAGAZINELUIZA
                                        WHERE
                                            ct.active=TRUE
                                        GROUP BY
                                            1 ) AS p
                                        ON
                                        p.seller_id=c.seller
                                        LEFT JOIN (
                                        SELECT
                                            maas_publicado,
                                            navigation_id
                                        FROM
                                            `maga-bigdata.indicadores_qa.fato_sku` ) AS q
                                        ON
                                        q.navigation_id=v.produtomarketplacekey

                                    """)


# Massivo para API


In [ ]:
def cluster_massivo(df) -> pd.DataFrame:
    """
    Parâmetro:
    _________________________
    df: Dataframe

    """
    def f_cluster(df):
        if df['cluster_de_delta'] == '0 a 1':
            return '5. Dispersão - abaixo de 2' 
        elif df['cluster_de_delta'] == '1 a 2':
            return '5. Dispersão - abaixo de 2'
        elif df['cluster_de_delta'] == '2 a 3':
            return '4. Dispersão - 2 a 7'
        elif df['cluster_de_delta'] == '3 a 4':
            return '4. Dispersão - 2 a 7'
        elif df['cluster_de_delta'] == '4 a 5':
            return '4. Dispersão - 2 a 7'
        elif df['cluster_de_delta'] == '5 a 6':
            return '4. Dispersão - 2 a 7'
        elif df['cluster_de_delta'] == '6 a 7':
            return '4. Dispersão - 2 a 7'
        elif df['cluster_de_delta'] == '7 a 8':
            return '3. Dispersão - acima 7'
        elif df['cluster_de_delta'] == '8 a 9':
            return '3. Dispersão - acima 7'
        elif df['cluster_de_delta'] == '10 a >':
            return '3. Dispersão - acima 7'
        else:
            return 'verificar'
    
    df['cluster_massivo'] = df.apply(f_cluster, axis=1)

    df["email_config_id"] = 22000015039
    df["cf_tipo"] = "Proteção de Marcas"
    df["type"] = "Indicadores de Qualidade"
    df["cf_market_place"] = "Orientação"
    df["priority"] = 4
    df["cf_especificao_indicadores_de_qualidade"] = "Canal de Denúncia / Hess"
    df["description"] = "Dispersão Frete"
    df["cf_origem_da_denncia"] = "Logística"
    df["group_id"] = 22000164158
    df["subject"] = "Denúncias Internas PS3P"
    df["status"] = 5
    df["email"] = "produtosuspeito3P@magazineluiza.com.br"
    #df["responder_id"] = 22025372385
    df["cf_orientao_para_comunicao"] = "Você precisa revisar seu catálogo e garantir que não haja erros no peso ou dimensão dos produtos."
    df["cf_denncia_vlida276276"] = "Sim (SKU despublicado)"

    return df





def massivo_final(df) -> pd.DataFrame:
    df.rename(columns={"cluster_massivo":"cf_subespecificao_indicadores_de_qualidade",
                       "seller":"cf_id_seller",
                       "produtomarketplacekey":"cf_id_magalu",
                       "title":"cf_nome_do_produto"}, inplace=True)
    
    df = df[['email_config_id',
            'cf_tipo',
            'type',
            'cf_market_place',
            'priority',
            'cf_especificao_indicadores_de_qualidade',
            'description',
            'cf_subespecificao_indicadores_de_qualidade',
            'cf_origem_da_denncia',
            'group_id',
            'cf_id_seller',
            'subject',
            'status',
            'email',
            'cf_id_magalu',
            'cf_nome_do_produto',
            'cf_orientao_para_comunicao',
            'cf_denncia_vlida276276']]

    return df



# function execution
df_massivo = df_bq[["seller",
                    "produtomarketplacekey",
                    "title",
                    "cluster_de_delta"]]

df_massivo = cluster_massivo(df_massivo)

df_massivo_final = massivo_final(df_massivo)



# API

In [ ]:
df_teste = df_massivo_final.query("cf_id_seller==['011monitoresoficial','3bkasa']").reset_index(drop=True)

con = CommunicationCreator('aAHXcBWAG4r8HUIGgtSt')
lista_dicionario = con.create_list_of_dictionaries(df_teste)
con.post_list_of_tickets(lista_dicionario)


# GFL Nets

In [ ]:
def gfl_nets(df) -> pd.DataFrame:
    """
    Parâmetro:
    _________________________
    df: Dataframe Original GFL
    """
    df_gfl_original = df
    df_gfl_original['val_pedido'] = df_gfl_original['pedido'].str.contains('NE|ZT')
    df_gfl_nets = df_gfl_original.query("val_pedido==True")
    df_gfl_nets['pedido'] = df_gfl_nets['pedido'].str[:-5]
    df_gfl_nets = df_gfl_nets.query('diferenca_frete>0.0 & seller!="magazineluiza"')
    df_gfl_nets.drop(columns=['val_pedido'], inplace=True)
    df_gfl_nets["pedido"] = df_gfl_nets["pedido"].str.strip()
    df_gfl_nets = cluster(df_gfl_nets)
    df_gfl_nets = df_gfl_nets.astype(str)
    df_gfl_nets.to_gbq(destination_table='marketplace_analytics.sku_cubagem_comunicado_correios_nets', project_id='maga-bigdata', if_exists='replace')
    return df_gfl_nets


# function execution
df_gfl_nets = gfl_nets(df_gfl_original)


In [ ]:
def query_bq_nets(project=str, query=str) -> pd.DataFrame:
    df_gfl_nets_final = dados.consulta_bigquery(project,query)
    return df_gfl_nets_final



# function execution
df_gfl_nets_final = ("maga-bigdata",
                                """
                                SELECT
                                    *
                                    FROM
                                    `marketplace_analytics.sku_cubagem_comunicado_correios_nets` AS s
                                    LEFT JOIN (
                                    SELECT
                                        DISTINCT shp.seller_id AS ID_SELLER,
                                        shp.seller_name AS LOJISTA,
                                        os.code AS COD_PEDIDO_LOJA,
                                        osh.delivery_id AS COD_PEDIDO,
                                        shp.product_name AS titulo,
                                        
                                    FROM
                                        `maga-bigdata.netshoes_dw.vw_order_full` os,
                                        UNNEST(os.order_shipping) osh,
                                        UNNEST(osh.shipping_product) shp,
                                        UNNEST(shipping_status_history) hist
                                    
                                    WHERE
                                        CAST(os.sale_date AS date) >= '2023-01-01'
                                        AND shp.source = '3P'
                                        AND osh.shipping_status_name IN ('CANCELADO',
                                        'DESPACHADO',
                                        'ENTREGUE',
                                        'FATURADO',
                                        'PRONTO PARA FATURAR')
                                    GROUP BY
                                        1,
                                        2,
                                        3,
                                        4,
                                        5) AS v
                                    ON
                                    v.COD_PEDIDO_LOJA=s.pedido
                                    AND v.ID_SELLER=seller
                            """)

# Análise Catalogo - Geral

In [ ]:
# Análise 2
df2 = dados.consulta_bigquery("maga-bigdata",
                              """
                              SELECT
                                DISTINCT S.SellerID,
                                S.Data_Publicado,
                                S.Plataforma,
                                S.Carteira,
                                H.status_loja,
                                c.qtd_sku_ativo,
                                ca.qtd_sku_tt,
                                e.media_preco,
                                vi.QTD_VISITAS,
                                v.GMV
                                FROM
                                `maga-bigdata.bi_ecomm.TB_SafraSellers` AS s
                                LEFT JOIN (
                                SELECT
                                    DISTINCT SellerID,
                                    Status AS status_loja
                                FROM
                                    `maga-bigdata.marketplace_analytics.view_dados_sellers_helena`
                                WHERE
                                    Status='Ativo') AS h
                                ON
                                s.SellerID=h.SellerID
                                LEFT JOIN (
                                SELECT
                                    seller_id,
                                    COUNT( DISTINCT sku) AS qtd_sku_ativo
                                FROM
                                    `maga-bigdata.catalogo.visao_catalogo_produto_atual`
                                WHERE
                                    active=TRUE
                                GROUP BY
                                    1 ) AS c
                                ON
                                c.seller_id=h.SellerID
                                LEFT JOIN (
                                SELECT
                                    DISTINCT SELLERID,
                                    ROUND(AVG(PRECO_POR),2) AS media_preco
                                FROM
                                    `maga-bigdata.bi_ecomm.TB_ESTOQUE_3P`
                                GROUP BY
                                    1) AS e
                                ON
                                e.SELLERID=h.SellerID
                                LEFT JOIN (
                                SELECT
                                    VCAP.IDSELLERMARKETPLACE AS IDSELLERMARKETPLACE,
                                    ROUND(SUM(ROUND(IFNULL(VCAP.VLVENDAITEM,0) - IFNULL(VCAP.VLDESCONTOAVISTA,0) + IFNULL(VCAP.VLJUROS,0) + IFNULL(VCAP.vlfreteclientenominal,0),2)),2) AS GMV,
                                FROM
                                    maga-bigdata.apolo.vw_fact_vendas_captadas VCAP
                                WHERE
                                    PARSE_DATE('%Y%m%d', CAST(datapedidokey AS STRING)) >= '2023-01-01'
                                    AND DATE(VCAP.datapagamentomktp) IS NOT NULL
                                    AND UPPER(VCAP.STATUS) <> 'TESTE'
                                    AND UPPER (IFNULL(VCAP.IDSELLERMARKETPLACE,'-1')) <> 'MAGAZINELUIZA'
                                GROUP BY
                                    1 ) AS v
                                ON
                                v.IDSELLERMARKETPLACE=h.SellerID
                                LEFT JOIN (
                                SELECT
                                    seller_id,
                                    COUNT( DISTINCT sku) AS qtd_sku_tt
                                FROM
                                    `maga-bigdata.catalogo.visao_catalogo_produto_atual`
                                GROUP BY
                                    1 ) AS ca
                                ON
                                ca.seller_id=h.SellerID
                                LEFT JOIN (
                                SELECT
                                    SELLER,
                                    COUNT( distinct SessionID) AS QTD_VISITAS
                                FROM
                                    `dp6-magazineluiza-gap.WA_DS_NAVEG_2.DS_GA_AJUSTADA`
                                WHERE
                                    DATE(DATE) >= '2023-01-01'
                                GROUP BY
                                    1 ) AS vi
                                ON
                                LOWER(vi.SELLER)=LOWER(h.SellerID)
                                WHERE
                                c.qtd_sku_ativo<=5
                                """)

df2['media_preco'] = df2['media_preco'].astype(float)
# Ingestão tabela drive
dados.grava_tabela_drive("base_geral","base",df2)

# Análise Frete Correios

In [ ]:
df_pedidos = dados.consulta_bigquery("maga-bigdata", 
                                        """
                                        WITH
                                            b_qtd_pedidos AS (
                                            SELECT
                                                numeropedidocliente,
                                                seller_id,
                                                COUNT(DISTINCT sku) AS qtd_sku,
                                            FROM
                                                `maga-bigdata.analytics_transport.dados_mle_cash`
                                            WHERE
                                                DATE(dt_pagamento_aprovado) >= CURRENT_DATE()-1
                                                AND flag_cancelamento = FALSE
                                                AND numeropedidocliente NOT LIKE ("%NE%")
                                                AND numeropedidocliente NOT LIKE ("%ZT%")
                                                AND numeropedidocliente NOT LIKE ("%ev%")
                                                AND numeropedidocliente NOT LIKE ("%KK%")
                                                AND numeropedidocliente NOT LIKE ("%CN%")
                                                AND numeropedidocliente NOT LIKE ("%LN%")
                                            GROUP BY
                                                1,
                                                2),

                                            b_pedidos_um_item AS (
                                            SELECT
                                                *
                                            FROM
                                                b_qtd_pedidos
                                            WHERE
                                                qtd_sku=1 )


                                            SELECT
                                            * except (numeropedidocliente_b,seller_id_b,qtd_sku_b)
                                            FROM
                                            b_pedidos_um_item AS bu
                                            LEFT JOIN (
                                            SELECT
                                                DISTINCT dt_pagamento_aprovado,
                                                numeropedidocliente as numeropedidocliente_b,
                                                seller_id as seller_id_b,
                                                malha,
                                                UF_origem,
                                                UF_destino,
                                                CONCAT(UF_origem,' - ',UF_destino) AS chave_uf,
                                                modalidade_entrega,
                                                shipping_service,
                                                sku,
                                                COUNT(DISTINCT sku) AS qtd_sku_b,
                                                ROUND(SUM(CAST(frete_cliente AS FLOAT64)),2) AS frete_cliente,
                                                ROUND(SUM(custo_rateio_peso),2) AS custo_rateio_peso,
                                                ROUND(SUM(CAST(spread AS FLOAT64)),2) AS spread,
                                                ROUND(sum(cast(peso as FLOAT64)),2) as peso
                                            FROM
                                                `maga-bigdata.analytics_transport.dados_mle_cash`
                                            WHERE
                                                DATE(dt_pagamento_aprovado) >= CURRENT_DATE()-1
                                                AND flag_cancelamento = FALSE
                                                AND numeropedidocliente NOT LIKE ("%NE%")
                                                AND numeropedidocliente NOT LIKE ("%ZT%")
                                                AND numeropedidocliente NOT LIKE ("%ev%")
                                                AND numeropedidocliente NOT LIKE ("%KK%")
                                                AND numeropedidocliente NOT LIKE ("%CN%")
                                                AND numeropedidocliente NOT LIKE ("%LN%")
                                            GROUP BY
                                                1,
                                                2,
                                                3,
                                                4,
                                                5,
                                                6,
                                                7,
                                                8,
                                                9,
                                                10) AS b
                                            ON
                                            bu.numeropedidocliente=b.numeropedidocliente_b
                                            AND bu.seller_id=b.seller_id_b

                                        """)


In [ ]:
df_pedidos.head()

In [ ]:
def calc_organizacao(df_pedidos:pd.DataFrame):
    if df_pedidos['modalidade_entrega']=="Retira loja" and df_pedidos['malha'] == "malha-fulfillment":
        return "FULFILLMENT RLE"
    elif df_pedidos['modalidade_entrega'] == "Retira loja":
        return "RLE" 
    elif df_pedidos['shipping_service']=="Ship From Seller":
        return "ULTRA"
    elif df_pedidos['malha'] == "malha-direta" and  df_pedidos['peso']>30.00: 
        return  "COLETA RODO"
    elif df_pedidos['malha'] == "malha-fulfillment" and df_pedidos['peso']>30.00:
        return "FULFILLMENT RODO"
    elif df_pedidos['malha'] == "malha-direta":
        return "MALHA DIRETA (GFL)"
    elif df_pedidos['malha'] == "malha-postagem":
        return "MALHA POSTAGEM (CORREIOS)"
    elif df_pedidos['malha'] =="malha-redespacho-cross":
        return "MALHA REDESPACHO (CORREIOS)"
    elif df_pedidos['malha']=="malha-fulfillment":
        return "FULFILLMENT COURIER"
    else:
        return ''

df_pedidos['organizacao'] = df_pedidos.apply(calc_organizacao, axis=1)

df_pedidos.count()

In [ ]:
# Base Etiquetas
df_etiqueta = dados.consulta_bigquery("maga-bigdata", 
                                      """
                                      SELECT
                                        --distinct
                                        --a.created_at,
                                        b.tracking_code   as Etiqueta,
                                        a.order_id        as pedido_cliente,
                                        a.seller as Seller
                                        --FORMAT_DATE('''%d/%m/%Y''', DATE (a.invoice_date)) as Dt_NF,
                                        --CASE WHEN a.price = 0 THEN 'SIM' ELSE 'N?O' END AS frete_gratis,
                                        --a.organization as Empresa,
                                        --(count(distinct a.order_id) over (partition by b.tracking_code)) as qtd
                                    from maga-bigdata.magaluentregas.shipping a
                                    left join maga-bigdata.magaluentregas.shipping_label b on b.shipping_id = a.id
                                    left join maga-bigdata.magaluentregas.shipping_campaign c on c.shipping_id = a.id
                                    where 
                                            date(a.created_at) >= '2023-01-01'
                                            and   b.tracking_code is not null
                                    union distinct
                                    select
                                        --a.created_at,
                                        r.tracking_code   as Etiqueta,
                                        a.order_id        as pedido_cliente,
                                        --FORMAT_DATE('''%d/%m/%Y''', DATE (a.invoice_date)) as Dt_NF,
                                        --CASE WHEN a.price = 0 THEN 'DEVOLU??O' ELSE 'DEVOLU??O' END AS frete_gratis,
                                        --null as Empresa,
                                        a.seller as Seller
                                        --(count(distinct a.order_id) over (partition by b.tracking_code)) as qtd
                                    from maga-bigdata.magaluentregas.shipping a
                                    left join maga-bigdata.magaluentregas.reverse_shipping_label r on r.reverse_shipping_id = a.id
                                    left join maga-bigdata.magaluentregas.shipping_campaign c on c.shipping_id = a.id
                                    where 
                                        date(a.created_at) >= '2023-01-01'
                                        and  r.tracking_code is not null
                                                                """)

In [ ]:
df_etiqueta.count()

In [ ]:
# Base Site Correios (buscar por API)
df_correios = pd.read_csv("/content/drive/MyDrive/dados_mle/Arquivos_correios/COM_9912427100_99999_25_05_2023_2_EXTRATO_PREVIA_CICLO_FC.CSV", skiprows=2, usecols=['Etiqueta', 'Valor do Servico']).drop_duplicates()
df_correios.head()